### Obtenção dos dados

In [31]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import plotly.express as px

def get_data():
    url = 'https://api.onedrive.com/v1.0/shares/s!Asuw4D2AHTOZlYc5XMBSbZydZF8WQw/root/content'
    resposta = requests.get(url)
    dados = StringIO(resposta.text)

    df_conteudo = pd.read_csv(dados, sep=';')
    #print(df_conteudo.head())

    url = 'https://api.onedrive.com/v1.0/shares/s!Asuw4D2AHTOZlYc7rk_HkK02f4T2GA/root/content'
    resposta = requests.get(url)
    dados = StringIO(resposta.text)

    df_play = pd.read_csv(dados, sep=';')
    #print(df_play.head())

    return df_play.merge(df_conteudo, on='id_conteudo', how='left')

df = get_data()
df

,id_user,id_conteudo,data,horas_consumidas,conteudo,categoria
0,150,10406,07/07/2019,"0,27",A,novela
1,139,10352,24/11/2019,"0,59",B,serie
2,182,10206,26/07/2019,"0,82",C,novela
3,199,10835,10/11/2019,"0,24",D,serie
4,185,10406,19/11/2019,"0,98",A,novela
5,144,10777,09/11/2019,"0,53",NaN,NaN
6,136,10206,18/07/2019,"0,07",C,novela
7,150,10835,09/07/2019,"0,61",D,serie
8,199,10406,24/07/2019,"0,70",A,novela
9,182,10352,08/08/2019,"0,61",B,serie


### Preparação dos dados

In [32]:
print(df.dtypes)
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

df['horas_consumidas'] = df['horas_consumidas'].str.replace(',', '.')
df['horas_consumidas'] = df['horas_consumidas'].astype(float)
print(df.dtypes)

id_user              int64
id_conteudo          int64
data                object
horas_consumidas    object
conteudo            object
categoria           object
dtype: object
id_user                      int64
id_conteudo                  int64
data                datetime64[ns]
horas_consumidas           float64
conteudo                    object
categoria                   object
dtype: object


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_user           25 non-null     int64         
 1   id_conteudo       25 non-null     int64         
 2   data              25 non-null     datetime64[ns]
 3   horas_consumidas  25 non-null     float64       
 4   conteudo          22 non-null     object        
 5   categoria         22 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 1.3+ KB


In [34]:
df[df['categoria'].isna()]

,id_user,id_conteudo,data,horas_consumidas,conteudo,categoria
5,144,10777,2019-11-09,0.53,NaN,NaN
10,199,10777,2019-09-17,0.42,NaN,NaN
14,199,10777,2019-08-19,0.82,NaN,NaN


In [35]:
df['categoria'] = df['categoria'].fillna('desconhecida')
df['conteudo']= df['conteudo'].fillna('desconhecido')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_user           25 non-null     int64         
 1   id_conteudo       25 non-null     int64         
 2   data              25 non-null     datetime64[ns]
 3   horas_consumidas  25 non-null     float64       
 4   conteudo          25 non-null     object        
 5   categoria         25 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 1.3+ KB


### 1 - Quantidade de horas consumidas e plays por categoria

In [36]:
df.groupby('categoria').agg(quantidade_horas_consumidas=('horas_consumidas', 'sum'),plays=('conteudo','count')).sort_values(by='quantidade_horas_consumidas', ascending=False).reset_index()

,categoria,quantidade_horas_consumidas,plays
0,serie,7.28,13
1,novela,3.71,9
2,desconhecida,1.77,3


### 2 - Ranking de novelas com mais horas consumidas por mês

In [37]:
df_novela = df[df['categoria'] == 'novela'].copy()
df_novela['mes_nome'] = df_novela['data'].dt.strftime('%B')
df_novela['mes_numero'] = df_novela['data'].dt.month

df_novela = df_novela.groupby(['categoria','conteudo','mes_nome','mes_numero']).agg({'horas_consumidas': 'sum'}).sort_values(by=['mes_numero', 'horas_consumidas'], ascending=[True, False]).reset_index()

df_novela

,categoria,conteudo,mes_nome,mes_numero,horas_consumidas
0,novela,A,July,7,1.13
1,novela,C,July,7,1.02
2,novela,A,October,10,0.58
3,novela,A,November,11,0.98


In [38]:
df_n =  df_novela.groupby(['categoria','conteudo','mes_nome','mes_numero']).agg({'horas_consumidas': 'sum'}).sort_values(by=['mes_numero', 'horas_consumidas'], ascending=[True, False]).reset_index().copy()
df_n.loc[df_n.groupby('mes_numero')['horas_consumidas'].idxmax()]

,categoria,conteudo,mes_nome,mes_numero,horas_consumidas
0,novela,A,July,7,1.13
2,novela,A,October,10,0.58
3,novela,A,November,11,0.98


### 3 - Conteúdo de primeiro play do usuário

Solução simples

In [39]:
df.sort_values('data').groupby('id_user').first().sort_values('data')

,id_conteudo,data,horas_consumidas,conteudo,categoria
id_user,,,,,
144,10406,2019-07-03,0.16,A,novela
150,10406,2019-07-07,0.27,A,novela
136,10206,2019-07-18,0.07,C,novela
185,10835,2019-07-21,0.51,D,serie
199,10406,2019-07-24,0.70,A,novela
182,10206,2019-07-26,0.82,C,novela
139,10352,2019-11-24,0.59,B,serie


Solução alternativa/complexa

In [40]:
df_primeiro_play = pd.DataFrame()

for user in df['id_user'].unique():
    primeiro_play = df[df['id_user'] == user]['data'].min()
    df_primeiro_play = pd.concat([df_primeiro_play,df[(df['id_user'] == user) & (df['data'] == primeiro_play)]])

df_primeiro_play.sort_values(by='data',inplace=True)
df_primeiro_play.reset_index(inplace=True,drop=True)

df_primeiro_play

,id_user,id_conteudo,data,horas_consumidas,conteudo,categoria
0,144,10406,2019-07-03,0.16,A,novela
1,150,10406,2019-07-07,0.27,A,novela
2,136,10206,2019-07-18,0.07,C,novela
3,185,10835,2019-07-21,0.51,D,serie
4,199,10406,2019-07-24,0.70,A,novela
5,182,10206,2019-07-26,0.82,C,novela
6,139,10352,2019-11-24,0.59,B,serie


### 4 - Minutos por play para cada usuário

In [41]:
df_minutos_per_play = df.copy()
df_minutos_per_play['minutos_consumidos'] = df_minutos_per_play['horas_consumidas'] * 60
df_minutos_per_play.groupby('id_user').agg({'minutos_consumidos': 'mean'}).sort_values(by='minutos_consumidos', ascending=False).reset_index()

,id_user,minutos_consumidos
0,182,47.40
1,139,35.40
2,185,33.15
3,199,32.64
4,144,29.90
5,150,20.20
6,136,17.40


### 5 - Qual a categoria mais consumida para cada usuário

In [42]:
df_categoria_favorita_usuario = df.groupby(['id_user','categoria']).agg({'horas_consumidas': 'sum'}).reset_index().copy()

df_categoria_favorita_usuario.sort_values(by='horas_consumidas', ascending=False, inplace=True)
df_categoria_favorita_usuario = df_categoria_favorita_usuario.groupby('id_user').first().reset_index().sort_values(by='horas_consumidas', ascending=False)

df_categoria_favorita_usuario

,id_user,categoria,horas_consumidas
2,144,serie,1.72
4,182,serie,1.55
6,199,desconhecida,1.24
5,185,serie,1.23
0,136,serie,0.80
3,150,serie,0.61
1,139,serie,0.59


### 6 - Conte uma história com os dados! Não precisa ser nada complexo. O objetivo é entendermos como você lida com informações e as analisa.

A partir dos dados disponíveis foi possível encontrar algumas tendências e padrões interessantes, que nos permite inferir algumas teorias para que com mais dados possam ser testadas e validadas.

---

O primeiro deles é de que aproximadamente entre agosto e outubro há uma queda acentuada nas horas
consumidas enquanto que em novembro há um aumento abrupto, evidênciado pelos gráficos abaixo.

In [43]:
title_font_size = 24
legend_title_font_size = 22
legend_font_size = 16
axes_title_font_size = 20
axes_font_size = 18

df = df.sort_values('data')

fig = px.line(df, x='data', y='horas_consumidas',color='categoria',
color_discrete_map={'novela': 'Crimson', 'serie': 'DeepSkyBlue','desconhecida':'GreenYellow'},symbol="categoria",
title='Horas consumidas por mês')

fig.update_xaxes(title='Mês', title_font=dict(size=axes_title_font_size), tickfont=dict(size=axes_font_size))
fig.update_yaxes(title='Horas Consumidas',title_font=dict(size=axes_title_font_size), tickfont=dict(size=axes_font_size))

fig.update_layout(
    title=dict(
        font=dict(
            size=title_font_size,
        ),
    ),
    legend=dict(
        title_font_size=legend_title_font_size,
        font=dict(
            size=legend_font_size,
        ),
    ),
)

fig.show()

In [44]:
df['mes_numero'] = df['data'].dt.month
df_horas_mes = df.copy().groupby(['mes_numero']).agg({'horas_consumidas': 'sum'}).reset_index().sort_values(by='mes_numero', ascending=True)

fig = px.line(df_horas_mes, x='mes_numero', y='horas_consumidas', title='Horas totais consumidas por mês')

fig.update_xaxes(title='Mês', title_font=dict(size=axes_title_font_size), tickfont=dict(size=axes_font_size))
fig.update_yaxes(title='Horas Consumidas',title_font=dict(size=axes_title_font_size), tickfont=dict(size=axes_font_size))

fig.update_traces(line=dict(color='Gold'))

fig.update_layout(
    title=dict(
        font=dict(
            size=title_font_size,
        ),
    ),
    legend=dict(
        title_font_size=legend_title_font_size,
        font=dict(
            size=legend_font_size,
        ),
    ),
)

fig.update_xaxes(range=[df_horas_mes['mes_numero'].min() - 0.25, df_horas_mes['mes_numero'].max() + 0.25])
fig.update_xaxes(
    tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    ticktext=['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
)
fig.show()

Tal comportamento aparenta ter alguma relação com períodos típicos de férias, o que faz sentido pois os usuários teriam mais tempo disponível para lazer e entreterimento. Considerando esse padrão de consumo, é recomendável que os servidores estejam dimensionados para suportar o aumento de tráfego no final do ano e aproveitar os meses de menor consumo para manutenções.

---

O segundo padrão interessante é de que a maioria dos usuários tiveram a seu primeiro play com novelas, provavelmente movidos pelo desejo de ver algum capítulo que não conseguiram assistir no horário de exibição, contudo, eles passaram a consumir mais séries do que novelas na plataforma, indicando que as séries desempenham grande papel na retenção de usuários. Afirmação reforçada pelo fato de que a maioria dos usuários assistiram mais horas de séries do que qualquer outra categoria.

In [45]:
fig = px.bar(df, x='data', y='horas_consumidas', color='categoria', barmode='group', facet_col='id_user',
color_discrete_map={'novela': 'Crimson', 'serie': 'DeepSkyBlue','desconhecida':'GreenYellow'})
fig.show()

Afirmação reforçada pelo fato de que a maioria dos usuários assistiram mais horas de séries do que qualquer outra categoria.

In [46]:
print(df_categoria_favorita_usuario)

   id_user     categoria  horas_consumidas
2      144         serie              1.72
4      182         serie              1.55
6      199  desconhecida              1.24
5      185         serie              1.23
0      136         serie              0.80
3      150         serie              0.61
1      139         serie              0.59


---

E por fim, a série D teve uma queda atípica se comparado com a tendência de horas de consumo, a sua queda de horas consumidas é menos abrupta do que dos demais conteúdos, porém, não há um aumento de consumo posteriormente, como ocorre com as demais séries. Isso pode indicar que a série D consegue manter a sua base de usuários, mas não consegue atrair novos usuários.

In [47]:
fig = px.line(df, x='data', y='horas_consumidas', title='Horas totais consumidas por mês',color='conteudo',
color_discrete_map={'A': 'HoneyDew', 'B': 'Ivory','C':'GhostWhite','D':'DeepSkyBlue','desconhecido':'Gainsboro'})


fig.update_xaxes(title='Data', title_font=dict(size=axes_title_font_size), tickfont=dict(size=axes_font_size))
fig.update_yaxes(title='Horas Consumidas',title_font=dict(size=axes_title_font_size), tickfont=dict(size=axes_font_size))

fig.update_layout(
    plot_bgcolor='gray',
    title=dict(
        font=dict(
            size=title_font_size,
        ),
    ),
    legend=dict(
        title_font_size=legend_title_font_size,
        font=dict(
            size=legend_font_size,
        ),
    ),
)

fig.show()